In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

import pygame
import time

import tkinter as tk
from tkinter import messagebox

In [17]:
def get_password():
    key_press_times = []
    key_hold_durations = []
    saved_times = []
    press_time = None
    last_event_time = None

    def on_key_press(event):
        nonlocal press_time, last_event_time
        current_time = time.time()
        if last_event_time is not None:
            key_press_times.append(current_time - last_event_time)
        press_time = current_time
        last_event_time = current_time

    def on_key_release(event):
        nonlocal press_time, last_event_time
        current_time = time.time()
        if press_time is not None:
            key_hold_durations.append(current_time - press_time)
        last_event_time = current_time

    def check_password():
        nonlocal saved_times, key_press_times, key_hold_durations, press_time, last_event_time
        password = password_entry.get()
        if password == "inetum":  # Replace "inetum" with the desired password
            saved_times = key_press_times + key_hold_durations
            messagebox.showinfo("Success", "Password correct!")
            root.destroy()  # Close the Tkinter window
        else:
            messagebox.showerror("Error", "Incorrect password")
            password_entry.delete(0, tk.END)  # Clear the password entry
            key_press_times.clear()
            key_hold_durations.clear()
            press_time = None
            last_event_time = None

    # Create the main window
    root = tk.Tk()
    root.title("Password Prompt")
    root.geometry("300x150")
    root.configure(bg='black')

    # Create a label
    label = tk.Label(root, text="Enter Password:", fg='white', bg='black', font=("Helvetica", 12))
    label.pack(pady=10)

    # Create a password entry
    password_entry = tk.Entry(root, show='*', fg='black', bg='white', font=("Helvetica", 12))
    password_entry.pack(pady=10)
    password_entry.bind("<KeyPress>", on_key_press)
    password_entry.bind("<KeyRelease>", on_key_release)

    # Create a button to submit the password
    submit_button = tk.Button(root, text="Submit", command=check_password, fg='white', bg='green', font=("Helvetica", 12))
    submit_button.pack(pady=10)

    # Start the main loop
    root.mainloop()

    return saved_times

def preprocessing(name, clase):
    df = pd.read_csv(f'../data/{name}.csv')
    time_data = {}
    attempts = df['attempt'].unique()

    for attempt in attempts:
        time_data['time_between_keys_attempt_'+str(attempt)] = df[df['attempt']==attempt]['timestamp']
        time_data['time_between_keys_attempt_'+str(attempt)] = list(time_data['time_between_keys_attempt_'+str(attempt)].diff().fillna(0).values)
    data  = [i[1:] for i in list(time_data.values())]
    df_preprocessed = pd.DataFrame(data,columns=['i','i-n','n','n-e','e','e-t','t','t-u','u','u-m','m'])
    df_preprocessed['y'] = clase
    return df_preprocessed

def split_df(df):
    df_letras = df[['i', 'n', 'e', 't', 'u', 'm']]
    df_espacios = df[['i-n', 'n-e', 'e-t', 't-u', 'u-m']]
    return (df_letras, df_espacios)

In [26]:
recorded_times = get_password()

df_preprocessed_nacho = preprocessing('keystroke_data_nacho', 0)
df_preprocessed_jorge = preprocessing('keystroke_data_jorge', 1)

df_test_nacho = preprocessing('keystroke_data_nacho_test', 0)
df_test_jorge = preprocessing('keystroke_data_jorge_test', 0)

data = df_preprocessed_nacho.iloc[:,:-1]

X = data.values

# Normalizar los datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

iso_forest = IsolationForest(contamination=0.055, random_state=42, n_estimators=300)
iso_forest.fit(X)

def es_mi_tiempo(nueva_serie):
    # Preprocesar nueva serie temporal
    nueva_serie = np.array(nueva_serie).reshape(1, -1)
    nueva_serie = scaler.transform(nueva_serie)
    
    # Predicción: -1 indica una anomalía, 1 indica que no es una anomalía
    prediccion = iso_forest.predict(nueva_serie)
    
    return prediccion[0] == 1

intento = es_mi_tiempo(recorded_times)

# Create the result pop-up window
result_root = tk.Tk()
result_root.withdraw()  # Hide the main window

if intento == False:
    tk.messagebox.showerror("Access Denied", "Usuario no autorizado", icon='error')
else:
    tk.messagebox.showinfo("Access Granted", "Usuario autorizado", icon='info')

# Close the result pop-up window
result_root.destroy()

ValueError: X has 12 features, but StandardScaler is expecting 11 features as input.